In [41]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import spacy

In [42]:
data = pd.read_csv('dataset.csv')

# Divide the data into training, validation and test
training_data, testing_data = train_test_split(data, test_size=0.2, random_state = 1)
# training_data, validation_data = train_test_split(training_data, test_size = 0.25, random_state = 1)

print("training data: ", training_data.shape)
# print("validation data: ", validation_data.shape)
print("testing data: ", testing_data.shape)

training data:  (6400, 2)
testing data:  (1600, 2)


In [43]:
nlp = spacy.load("sv_core_news_sm")

def preprocess(question):
    doc = nlp(question)
    entities = [ent.text for ent in doc.ents]
    return entities

In [44]:
# Find the maximum number of entities in the training dataset
max_entities = max([len(preprocess(row["question"])) for _, row in data.iterrows()])

In [45]:
tree = DecisionTreeClassifier()

# Load the training dataset
X_train = []
y_train = []

# Preprocess each question in the training dataset and append the resulting entities to X_train
for _, row in training_data.iterrows():
    entities = preprocess(row["question"])
    padded_entities = entities + ['0'] * (max_entities - len(entities))
    X_train.append(padded_entities)
    y_train.append(row["label"])


In [46]:
# One-hot encode the entity lists using scikit-learn's MultiLabelBinarizer
mlb = MultiLabelBinarizer()
X_train = mlb.fit_transform(X_train)

In [47]:
# Train the decision tree model
tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [48]:
# Load the test dataset
X_test = []
y_test = []

# Preprocess each question in the test dataset and append the resulting entities to X_test
for _, row in testing_data.iterrows():
    entities = preprocess(row["question"])
    padded_entities = entities + ['0'] * (max_entities - len(entities))
    X_test.append(padded_entities)
    y_test.append(row["label"])

In [49]:
# One-hot encode the entity lists using scikit-learn's MultiLabelBinarizer
X_test = mlb.transform(X_test)

In [50]:
# Test the decision tree model
y_pred = tree.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

ValueError: X has 646 features, but DecisionTreeClassifier is expecting 1931 features as input.